<a href="https://colab.research.google.com/github/DDDS18-GTFS/ddds.18.capstone/blob/dev.Andrew/ABQ_trial_RT_Diagnostics_and_TheMachine_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Capture the ABQ Data

In [ ]:
#This Cell will capture the data to use for later analysis

import requests
import pandas as pd
import json
import time
import os
from datetime import datetime

# ✅ Albuquerque RT feed URL
GTFS_URL = "https://data.cabq.gov/transit/realtime/route/allroutes.json"

# 🕒 Configuration
NUM_SNAPSHOTS = 10           # Try 10 for ~5 minutes @ 30s
SLEEP_SECONDS = 30
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M")
OUTPUT_DIR = f"/mnt/data/cabq_snapshots_{RUN_TAG}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"🚍 Starting Albuquerque data collection ({NUM_SNAPSHOTS} snapshots)...")

# 📸 Define the fetch function
def fetch_cabq_snapshot_fixed(snapshot_id):
    url = GTFS_URL
    response = requests.get(url)
    data = json.loads(response.content.decode("iso-8859-1"))
    timestamp_collected = datetime.utcnow().isoformat()

    records = []
    for vehicle in data.get("allroutes", []):
        records.append({
            "snapshot_id": snapshot_id,
            "timestamp_collected": timestamp_collected,
            "vehicle_id": vehicle.get("vehicle_id"),
            "latitude": vehicle.get("latitude"),
            "longitude": vehicle.get("longitude"),
            "heading": vehicle.get("heading"),
            "speed_mph": vehicle.get("speed_mph"),
            "route_short_name": vehicle.get("route_short_name"),
            "trip_id": vehicle.get("trip_id"),
            "next_stop_id": vehicle.get("next_stop_id"),
            "next_stop_name": vehicle.get("next_stop_name"),
            "next_stop_sched_time": vehicle.get("next_stop_sched_time")
        })

    return pd.DataFrame(records)

# 📦 Collect and store all snapshots
all_snapshots = []

for i in range(NUM_SNAPSHOTS):
    try:
        print(f"\n📸 Snapshot {i+1}/{NUM_SNAPSHOTS} at {datetime.now().strftime('%H:%M:%S')}")
        df_snapshot = fetch_cabq_snapshot_fixed(i + 1)
        all_snapshots.append(df_snapshot)
        print(f"✅ Collected {len(df_snapshot)} vehicles")

        if i < NUM_SNAPSHOTS - 1:
            time.sleep(SLEEP_SECONDS)

    except Exception as e:
        print(f"❌ Error during snapshot {i+1}: {e}")
        continue

# 🧮 Combine all snapshots
df_all = pd.concat(all_snapshots, ignore_index=True)

# 💾 Save to CSV and Parquet
csv_path = os.path.join(OUTPUT_DIR, f"cabq_gtfs_snapshots_{RUN_TAG}.csv")
parquet_path = os.path.join(OUTPUT_DIR, f"cabq_gtfs_snapshots_{RUN_TAG}.parquet")

df_all.to_csv(csv_path, index=False)
df_all.to_parquet(parquet_path, index=False)

print(f"\n✅ Saved {len(df_all)} total vehicle records")
print(f"📄 CSV: {csv_path}")
print(f"📦 Parquet: {parquet_path}")


🚍 Starting Albuquerque data collection (10 snapshots)...

📸 Snapshot 1/10 at 21:11:10
✅ Collected 340 vehicles


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1-4081979907.py", line 59, in <cell line: 0>
    time.sleep(SLEEP_SECONDS)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
           ^^^^^^^^^^^^^^^^^^^^^^^

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_all.head()
df_all['vehicle_id'].nunique()
df_all.groupby('snapshot_id').size()


#Tuning the Event Info

Some quick calculations to try to capture relevant events
- A relevant event is being set to a greater than average distance and time between stops, with a movement from one stop to another being used as a proxy for "relevant"
*******
The average speed of Albuquerque buses based on a sampled 5-minute GTFS snapshot is approximately:
- 6.37 km/h (kilometers per hour)
- 3.96 mph (miles per hour)
- These speeds are consistent with low-traffic stop-and-go urban transit patterns during short observation windows.
******
While the exact average distance between all bus stops in Albuquerque isn't immediately available, general guidelines and related statistics provide some insight:
- General Spacing Recommendations: Transit and bus stop design guidelines suggest an average stop spacing of 1,300 feet for local fixed routes in areas with higher population and employment density.
- Research Findings: Studies of bus stop spacing in the United States have found the overall mean spacing to be 313 meters, which is approximately 1027 feet. Another source mentions that buses often stop as frequently as every one or two blocks, with an average of 805 feet [245 m] between stops.
- ABQ Specific Context: ABQ Ride is currently undergoing a network plan review to ensure transit services reflect community priorities, including balancing ridership and coverage needs.
*******
Average time between ABQ Ride bus stops
- While there isn't a readily available precise average time between each individual bus stop on every route in Albuquerque, some information can help understand the general spacing and frequency of ABQ Ride buses:
- Frequency Varies: ABQ Ride offers several types of routes including Rapid Ride (now ARTx, effectively), regular, commuter, and BRT (ART). The frequency of buses varies significantly depending on the specific route and time of day, ranging from as frequent as every 15 minutes to as infrequent as once per hour. - The City of Albuquerque (.gov) ABQ Ride Forward Network Plan indicates that route frequencies in their proposed recovery network range from 15 minutes (red routes) to 60 minutes (light blue routes).
- ART/BRT Faster: The ART (Albuquerque Rapid Transit) system, running along Central Avenue, is known for being relatively fast and reliable.
- Potential Delays: It's important to be aware that even with scheduled frequencies, buses can experience delays due to factors like traffic, rider numbers, and potential issues with the bus itself. Some users have reported significant delays, especially outside of peak hours or with routes that run less frequently.


In [ ]:
# ---------------------
# Threshold parameters
# ---------------------

# Distance (in degrees) that counts as a GPS "jump" (this is greater than the average stop distance)
JUMP_DISTANCE_THRESHOLD = 0.005  # ≈ 500 meters

# Time gap (in seconds) that counts as a disappearance
DISAPPEARANCE_TIME_THRESHOLD = 300  # 5 minutes

# Only show vehicles with at least this many jumps
MIN_JUMP_COUNT_PER_VEHICLE = 1

# Number of vehicles to sample for map clarity
NUM_VEHICLES_TO_SAMPLE = 200


##STEP 1: LOAD & PREPARE DATA

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Load your dataset
RUN_TAG = "20250721_0126"
csv_path = f"/content/cabq_gtfs_snapshots_{RUN_TAG}.csv"
df_all = pd.read_csv(csv_path)
df_all["timestamp_collected"] = pd.to_datetime(df_all["timestamp_collected"])
df_all.sort_values(by=["vehicle_id", "timestamp_collected"], inplace=True)

# Filter for vehicles with at least 3 data points
vehicle_counts = df_all["vehicle_id"].value_counts()
vehicles_ok = vehicle_counts[vehicle_counts >= 3].index

# Sample a manageable number of vehicles
sample_vehicles = np.random.choice(vehicles_ok, size=NUM_VEHICLES_TO_SAMPLE, replace=False)
df_sample = df_all[df_all["vehicle_id"].isin(sample_vehicles)].copy()

# 🧾 Check data
print("✅ Loaded GTFS snapshot data")
print("📊 Shape:", df_all.shape)
print("🕒 Timestamp range:", df_all['timestamp_collected'].min(), "to", df_all['timestamp_collected'].max())
df_all.head()

✅ Loaded GTFS snapshot data
📊 Shape: (13560, 12)
🕒 Timestamp range: 2025-07-21 01:26:49.211366 to 2025-07-21 01:46:26.391846


,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_name,next_stop_sched_time
17,1,2025-07-21 01:26:49.211366,351,35.089151,-106.734347,313.0,0.0,Off Duty,0,0,No Data,14:46:07
356,2,2025-07-21 01:27:19.385662,351,35.089151,-106.734347,313.0,0.0,Off Duty,0,0,No Data,14:46:07
695,3,2025-07-21 01:27:49.564446,351,35.089151,-106.734347,313.0,0.0,Off Duty,0,0,No Data,14:46:07
1034,4,2025-07-21 01:28:19.745104,351,35.089151,-106.734347,313.0,0.0,Off Duty,0,0,No Data,14:46:07
1372,5,2025-07-21 01:28:49.929879,351,35.089151,-106.734347,313.0,0.0,Off Duty,0,0,No Data,14:46:07


##STEP 2: CALCULATE JUMPS & TIME GAPS

In [ ]:
# Compute diffs for position and time
df_sample["lat_diff"] = df_sample.groupby("vehicle_id")["latitude"].diff()
df_sample["lon_diff"] = df_sample.groupby("vehicle_id")["longitude"].diff()
df_sample["jump_dist"] = (df_sample["lat_diff"]**2 + df_sample["lon_diff"]**2)**0.5

df_sample["time_diff"] = df_sample.groupby("vehicle_id")["timestamp_collected"].diff().dt.total_seconds()

# Label events
df_sample["is_jump"] = df_sample["jump_dist"] > JUMP_DISTANCE_THRESHOLD
df_sample["is_disappearance"] = df_sample["time_diff"] > DISAPPEARANCE_TIME_THRESHOLD


##STEP 3: FILTER & ORGANIZE ANOMALIES

In [ ]:
# Get jump rows and enrich with previous position
jumps_df = df_sample[df_sample["is_jump"]].copy()
jumps_df["lat_prev"] = df_sample.groupby("vehicle_id")["latitude"].shift()
jumps_df["lon_prev"] = df_sample.groupby("vehicle_id")["longitude"].shift()

# Filter for vehicles with sufficient jumps
jump_counts = jumps_df["vehicle_id"].value_counts()
keep_jumpers = jump_counts[jump_counts >= MIN_JUMP_COUNT_PER_VEHICLE].index
jumps_df = jumps_df[jumps_df["vehicle_id"].isin(keep_jumpers)]

# Recalculate filtered set for map
df_sample = df_sample[df_sample["vehicle_id"].isin(keep_jumpers)].copy()

# Disappearances and reappearances
disappear_df = df_sample[df_sample["is_disappearance"] == True].copy()
reappear_df = df_sample[df_sample["is_disappearance"].shift(-1) == True].copy()


##STEP 4: PLOT WITH FOLIUM

In [ ]:
import folium

m = folium.Map(location=[35.0844, -106.6504], zoom_start=12)

# -- Plot jump lines and markers --
for _, row in jumps_df.iterrows():
    start = [row["lat_prev"], row["lon_prev"]]
    end = [row["latitude"], row["longitude"]]

    # Line between jumps
    folium.PolyLine(
        locations=[start, end],
        color="orange", weight=2,
        tooltip=f"{row['vehicle_id']} jump"
    ).add_to(m)

    # Start marker (blue)
    folium.CircleMarker(
        location=start,
        radius=4, color="blue", fill=True, fill_opacity=0.9,
        tooltip=f"{row['vehicle_id']}_start"
    ).add_to(m)

    # End marker (purple)
    folium.CircleMarker(
        location=end,
        radius=4, color="purple", fill=True, fill_opacity=0.9,
        tooltip=f"{row['vehicle_id']}_end"
    ).add_to(m)

# -- Disappearances (red X) --
for _, row in disappear_df.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        icon=folium.Icon(color="red", icon="times-circle", prefix="fa"),
        tooltip=f"{row['vehicle_id']} disappeared"
    ).add_to(m)

# -- Reappearances (green check) --
for _, row in reappear_df.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        icon=folium.Icon(color="green", icon="check-circle", prefix="fa"),
        tooltip=f"{row['vehicle_id']} reappeared"
    ).add_to(m)

m


#Looking at adding Static GTFS Info

In [ ]:
import zipfile
import pandas as pd

# Path to your GTFS static ZIP
gtfs_zip_path = "/content/google_transit.zip"

# Extract GTFS files
with zipfile.ZipFile(gtfs_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs_static")

# Load core GTFS tables
routes = pd.read_csv("/content/gtfs_static/routes.txt")
trips = pd.read_csv("/content/gtfs_static/trips.txt")
stop_times = pd.read_csv("/content/gtfs_static/stop_times.txt")
stops = pd.read_csv("/content/gtfs_static/stops.txt")
calendar = pd.read_csv("/content/gtfs_static/calendar.txt")

# Optional: Check shapes
for name, df in [("routes", routes), ("trips", trips), ("stop_times", stop_times), ("stops", stops), ("calendar", calendar)]:
    print(f"{name}: {df.shape}")


routes: (23, 9)
trips: (2139, 10)
stop_times: (100069, 10)
stops: (1814, 12)
calendar: (4, 10)


In [ ]:
snapshot = pd.read_csv("/content/cabq_gtfs_snapshots_20250721_0126.csv")
snapshot.head()


,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_name,next_stop_sched_time
0,1,2025-07-21T01:26:49.211366,610,35.147320,-106.639650,186.0,9.65604,10,0,0,4th @ Guadalupe,11:32:57
1,1,2025-07-21T01:26:49.211366,383,35.058513,-106.569910,17.0,0.00000,16,601256,2801,Gibson @ San Pedro,15:18:45
2,1,2025-07-21T01:26:49.211366,627,35.106680,-106.550930,178.0,0.00000,31,0,0,Wyoming @ Northeastern,13:03:07
3,1,2025-07-21T01:26:49.211366,459,35.106075,-106.706482,246.0,61.00000,5,601701,1729,To Garage,17:52:12
4,1,2025-07-21T01:26:49.211366,961,35.089586,-106.732609,294.0,0.00000,54,0,0,A.T.C. (Bay L),06:21:00


In [ ]:
snapshot["trip_id"] = snapshot["trip_id"].astype(str)
trips["trip_id"] = trips["trip_id"].astype(str)

snapshot_with_trips = snapshot.merge(trips, on="trip_id", how="left")


In [ ]:
routes["route_id"] = routes["route_id"].astype(str)
snapshot_with_trips["route_id"] = snapshot_with_trips["route_id"].astype(str)

snapshot_full = snapshot_with_trips.merge(routes, on="route_id", how="left")


##🛠 Tip: Always Normalize GTFS Join Columns
Here's a quick checklist of join keys and typical types to standardize:

| Column       | Target DataFrames                     | Recommended Type |
| ------------ | ------------------------------------- | ---------------- |
| `trip_id`    | `snapshot`, `trips`                   | `str`            |
| `route_id`   | `trips`, `routes`                     | `str`            |
| `stop_id`    | `snapshot`, `stops`, `stop_times`     | `str`            |
| `service_id` | `trips`, `calendar`, `calendar_dates` | `str`            |


#Validation check

✅ 1. Merge Completeness
We want to confirm that:

All snapshot rows still exist after the merge

No major fields from the trips or routes tables are missing (i.e., not matched)

Check Row Count:

(All three counts should match if merges were successful.)

In [ ]:
print("Original snapshot rows:", len(snapshot))
print("After merging with trips:", len(snapshot_with_trips))
print("After merging with routes:", len(snapshot_full))


Original snapshot rows: 13560
After merging with trips: 13560
After merging with routes: 13560


Check for Unmatched Trip IDs:

In [ ]:
missing_trips = snapshot_with_trips["route_id"].isnull().sum()
print("Snapshot rows missing trip match:", missing_trips)


Snapshot rows missing trip match: 0


Check for Unmatched Route IDs:

(If either is greater than 0, we should inspect the join keys and data formatting.)


In [ ]:
missing_routes = snapshot_full["route_long_name"].isnull().sum()
print("Snapshot rows missing route match:", missing_routes)


Snapshot rows missing route match: 13560


✅ 2. Sanity Sample

See a few enriched records to verify static fields are present:

(You should see real route info (not NaNs) and consistent route_id + trip_id.)

In [ ]:
snapshot_full[[
    "vehicle_id",
    "timestamp",
    "trip_id",
    "route_id",
    "route_short_name",
    "route_long_name",
    "service_id"
]].sample(5)


KeyError: "['timestamp', 'route_short_name'] not in index"

In [ ]:
print(snapshot_full.columns.tolist())


['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude', 'longitude', 'heading', 'speed_mph', 'route_short_name_x', 'trip_id', 'next_stop_id', 'next_stop_name', 'next_stop_sched_time', 'route_id', 'service_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed', 'agency_id', 'route_short_name_y', 'route_long_name', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color']


In [ ]:
print(snapshot_with_trips.columns.tolist())  # should include 'route_id'
print(routes.columns.tolist())               # should include 'route_id' and 'route_short_name'


['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude', 'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id', 'next_stop_id', 'next_stop_name', 'next_stop_sched_time', 'route_id', 'service_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed']
['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color']


In [ ]:
print(snapshot_with_trips["route_id"].dtype)
print(routes["route_id"].dtype)


object
object


✅ 3. Null Summary

Let’s check how many rows have missing values in static fields:

(If these are mostly filled, your integration is sound.)

In [ ]:
snapshot_full[["route_id", "route_short_name", "route_long_name"]].isnull().sum()


KeyError: "['route_short_name'] not in index"

#Redefining code names

✅ Step 1: Define Canonical Column Roles

We'll define the roles we care about, then scan the actual column names to identify matches.

In [ ]:
column_roles = {
    "timestamp": ["timestamp_collected", "timestamp", "vehicle_timestamp", "recorded_time"],
    "vehicle_id": ["vehicle_id", "vid", "bus_id"],
    "trip_id": ["trip_id"],
    "route_id": ["route_id"],
    "route_short_name": ["route_short_name", "route_short_name_y", "trip_route_short_name"],
    "route_long_name": ["route_long_name"],
    "latitude": ["latitude", "lat"],
    "longitude": ["longitude", "lon", "lng"],
}


✅ Step 2: Match Actual Columns to Roles


In [ ]:
def resolve_column_mapping(df, role_dict):
    mapping = {}
    for role, candidates in role_dict.items():
        for col in candidates:
            if col in df.columns:
                mapping[role] = col
                break
        else:
            mapping[role] = None  # Mark as not found
    return mapping

column_map = resolve_column_mapping(snapshot_full, column_roles)
print(column_map)


{'timestamp': 'timestamp_collected', 'vehicle_id': 'vehicle_id', 'trip_id': 'trip_id', 'route_id': 'route_id', 'route_short_name': 'route_short_name_y', 'route_long_name': 'route_long_name', 'latitude': 'latitude', 'longitude': 'longitude'}


✅ Step 3: Use Column Map in Flexible Queries

Now you can sample adaptively:

In [ ]:
cols_to_display = [
    column_map['vehicle_id'],
    column_map['timestamp'],
    column_map['trip_id'],
    column_map['route_id'],
    column_map['route_short_name'],
    column_map['route_long_name'],
]

snapshot_full[cols_to_display].sample(5)


,vehicle_id,timestamp_collected,trip_id,route_id,route_short_name_y,route_long_name
6508,626,2025-07-21T01:36:22.683291,Undetermined,nan,NaN,NaN
3689,4101,2025-07-21T01:31:51.002141,Undetermined,nan,NaN,NaN
9225,634,2025-07-21T01:40:24.135451,Undetermined,nan,NaN,NaN
7755,3005,2025-07-21T01:37:53.230425,0,nan,NaN,NaN
8534,619,2025-07-21T01:39:23.775832,0,nan,NaN,NaN


Not really happy with this output

#✅ Diagnostics


In [ ]:
#How many snapshot trip_ids are found in trips.txt?

found_in_trips = snapshot["trip_id"].isin(trips["trip_id"].astype(str)).sum()
print(f"{found_in_trips} of {len(snapshot)} trip_ids matched to static GTFS")


457 of 13560 trip_ids matched to static GTFS


That’s a critical red flag:

Only 457 of 13,560 trip_ids (~3.4%) in your snapshot match static GTFS.

This suggests a serious mismatch between your snapshot data and the static GTFS you're using.

🔍 Likely Causes (Ranked by Probability)
1. Your GTFS Static Data is Out of Date
Your snapshot is from 2025-07-21

Your static GTFS may be older than that (e.g., from June or earlier)

Trip IDs are generated per service calendar — a stale trips.txt won’t contain active IDs

2. GTFS Static Source Doesn’t Match Snapshot Source
For example, snapshot may come from a different provider (TransitLand, proprietary feed, etc.)

Even for ABQ RIDE, the version of the static feed used in Transitland may not match the feed used by whoever generated the snapshot

3. trip_id Normalization Issue (Unlikely Now)
You already cast trip_id to string, but some feeds still embed formatting (e.g., trip_id=12345_ABC123) vs raw integers.



##Diagnostic Actions

In [ ]:
#A. Inspect Sample of Unmatched trip_id Values

unmatched_ids = snapshot[~snapshot["trip_id"].isin(trips["trip_id"].astype(str))]
print(unmatched_ids["trip_id"].drop_duplicates().head(10).to_list())

['0', 'Undetermined']


Do they look like:

"Undetermined" / "0"? (Junk)

Long encoded strings? ("12345_ABC20250721")

Structured differently than static trips.txt?

🔹 A. trip_id Sample
['0', 'Undetermined']

✅ Confirms that most unmatched trip_ids are invalid placeholders in the snapshot. You are not missing real IDs — the issue is bad data in the real-time source, not the static feed.

In [ ]:
#B. Compare Date Ranges in Static Feed

calendar.start_date = pd.to_datetime(calendar.start_date, format="%Y%m%d")
calendar.end_date = pd.to_datetime(calendar.end_date, format="%Y%m%d")

calendar_summary = calendar[["start_date", "end_date"]].agg(["min", "max"])
print(calendar_summary)

#If end_date is before 2025-07-21, that confirms you're using an outdated static feed.

    start_date   end_date
min 2025-04-05 2025-08-22
max 2025-04-05 2025-08-22


🔹 B. Static GTFS Calendar
start_date = 2025-04-05, end_date = 2025-08-22

✅ Your static GTFS is valid and current for 2025-07-21.

📌 So the low trip match rate is not due to feed staleness.



In [ ]:
#C. Count Valid trip_ids in Static for 2025-07-21

target_date = pd.to_datetime("2025-07-21")

# Only trips active on this date
valid_service_ids = calendar[
    (calendar.start_date <= target_date) &
    (calendar.end_date >= target_date)
]["service_id"]

trips_valid_on_date = trips[trips["service_id"].isin(valid_service_ids)]
print("Trips active on 2025-07-21:", len(trips_valid_on_date))

#If trips_valid_on_date is empty or tiny, that confirms outdated static data.

Trips active on 2025-07-21: 2113


🔹 C. Trips Active on 2025-07-21
2113 trips are valid in the static feed for that date.

✅ Plenty of trips to work with — the problem is that the snapshot data is mostly populated with garbage trip IDs (not a mismatch between the datasets).

In [ ]:
#How many route_ids in snapshot match routes.txt?

joined_route_ids = snapshot_with_trips["route_id"].dropna().unique()
static_route_ids = routes["route_id"].astype(str).unique()

missing_routes = set(joined_route_ids) - set(static_route_ids)
print("Unmatched route_ids:", missing_routes)


Unmatched route_ids: {'nan', '4309.0', '4330.0', '4321.0', '4328.0', '4331.0'}


🔹 D. Unmatched route_ids
{‘4309.0’, ‘4330.0’, ‘4321.0’, ‘4328.0’, ‘4331.0’}

This means:

route_id came into your snapshot as float values (note .0)

But in GTFS static, they are likely strings, like "4330" (not "4330.0")

#Immediate Fixes

In [ ]:
#✅ 1. Clean Trip ID Field
snapshot["trip_id"] = snapshot["trip_id"].astype(str)
snapshot_clean = snapshot[~snapshot["trip_id"].isin(["0", "Undetermined", "", "nan", "None"])]


In [ ]:
#✅ 2. Normalize and Fix route_id Formatting
#If route_id came through as float, fix it like this:
snapshot_clean["route_id"] = snapshot_clean["route_id"].astype(str).str.replace(r"\.0$", "", regex=True)
routes["route_id"] = routes["route_id"].astype(str)
#Then re-run the merge with routes.txt.

In [ ]:
#✅ 3. Re-run the Integration Pipeline
# Merge trips again
trips["trip_id"] = trips["trip_id"].astype(str)
snapshot_with_trips = snapshot_clean.merge(trips, on="trip_id", how="left")

# Merge routes again
snapshot_with_trips["route_id"] = snapshot_with_trips["route_id"].astype(str).str.replace(r"\.0$", "", regex=True)
routes["route_id"] = routes["route_id"].astype(str)
snapshot_full = snapshot_with_trips.merge(routes, on="route_id", how="left")


In [ ]:
#🔍 Post-Integration Metrics to Recheck
print("Total snapshot rows after cleaning:", len(snapshot_clean))
print("Trip match success:", snapshot_with_trips["route_id"].notnull().sum())
print("Route match success:", snapshot_full["route_long_name"].notnull().sum())
#This will help you assess how many rows are fully grounded in static GTFS.

Total snapshot rows after cleaning: 457
Trip match success: 457
Route match success: 457


##🧱 Recommended Next Step: Create data_quality Column
To future-proof this for incoming snapshots:

In [ ]:
def classify_row(row):
    if row["trip_id"] in ["0", "Undetermined", "", "nan", "None"]:
        return "invalid_trip_id"
    elif pd.isna(row["route_id"]) or row["route_id"] == "nan":
        return "missing_route_id"
    elif pd.isna(row["route_long_name"]):
        return "missing_route_metadata"
    else:
        return "valid"

snapshot_full["data_quality"] = snapshot_full.apply(classify_row, axis=1)
snapshot_full["data_quality"].value_counts()


,count
data_quality,
valid,457


#Checking Gabriel's *The Machine* captures

In [2]:
#✅ Step 1: Load and Inspect the New Snapshot
import pandas as pd

new_snapshot_path = "/content/cabq_gtfs_snapshots_20250722_1415.csv"
df_new = pd.read_csv(new_snapshot_path)
df_new.info()
df_new.head(3)

#Also print the columns:
print(df_new.columns.tolist())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40460 entries, 0 to 40459
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   snapshot_id           40460 non-null  int64  
 1   timestamp_collected   40460 non-null  object 
 2   vehicle_id            40460 non-null  int64  
 3   latitude              40460 non-null  float64
 4   longitude             40460 non-null  float64
 5   heading               40460 non-null  float64
 6   speed_mph             40460 non-null  float64
 7   route_short_name      40460 non-null  object 
 8   trip_id               40460 non-null  object 
 9   next_stop_id          40460 non-null  object 
 10  next_stop_name        40460 non-null  object 
 11  next_stop_sched_time  40460 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 3.7+ MB
['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude', 'longitude', 'heading', 'speed_mph', 'route_short_

✅ Summary of Findings (with Fixes)

🔹 Step 1: Structure & Columns
- Metric	Value
- Rows	40,460
- Columns	12 (core fields only)
- trip_id	Present, all strings ✅
- route_id	❌ Missing entirely
- route_short_name	Present ✅

📌 Key Gap: No route_id present. That breaks the trips → routes chain we used in the previous pipeline.



In [3]:
#✅ Step 2: Trip ID Validity
df_new["trip_id"] = df_new["trip_id"].astype(str)
invalid_trip_ids = df_new["trip_id"].isin(["0", "Undetermined", "nan", "", "None"]).sum()
total_rows = len(df_new)

print(f"Total rows: {total_rows}")
print(f"Invalid trip_ids: {invalid_trip_ids}")
print(f"Percent valid: {100 * (total_rows - invalid_trip_ids) / total_rows:.2f}%")


Total rows: 40460
Invalid trip_ids: 33075
Percent valid: 18.25%


🔹 Step 2: trip_id Validity
- Invalid trip_ids: 33,075 → only 18.25% valid

🟡 That's a significant improvement from 3.4% in the last snapshot, but still low.
- Still likely due to real-time feed instability or partial trip assignments.

In [4]:
#✅ Step 3: Route ID Normalization (Preview)
# We'll also check route formatting now that we know floats were a problem previously:

df_new["route_id"] = df_new["route_id"].astype(str).str.replace(r"\.0$", "", regex=True)
print(df_new["route_id"].dropna().unique()[:10])


KeyError: 'route_id'

🔹 Step 3: route_id Access Fails
- ❌ KeyError: 'route_id'
This means this snapshot does not include route_id at all.

So you'll have to:

- Get route_id from a merge with static GTFS trips.txt, using trip_id

- Then (optionally) use route_id to merge with routes.txt

In [5]:
#✅ Step 4: Compare Columns to Prior Snapshot

# Assuming you still have previous snapshot loaded as `snapshot`
print("Previous columns:", set(snapshot.columns))
print("New columns:     ", set(df_new.columns))

missing_in_new = set(snapshot.columns) - set(df_new.columns)
new_extras = set(df_new.columns) - set(snapshot.columns)

print("Missing columns in new snapshot:", missing_in_new)
print("New extra columns in new snapshot:", new_extras)


NameError: name 'snapshot' is not defined

🔹 Step 4: snapshot is undefined

Expected — you're in a fresh notebook.

#Fixes and next steps

In [6]:
#✅ Step 1: Reload Static GTFS and Normalize It
import pandas as pd
import zipfile

# Adjust if needed — make sure this is the static feed aligned with 2025-07-22
gtfs_zip_path = "/content/google_transit.zip"

with zipfile.ZipFile(gtfs_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs_static")

trips = pd.read_csv("/content/gtfs_static/trips.txt", dtype=str)
routes = pd.read_csv("/content/gtfs_static/routes.txt", dtype=str)


In [7]:
#✅ Step 2: Load and Clean New Snapshot
df_new = pd.read_csv("/content/cabq_gtfs_snapshots_20250722_1415.csv")
df_new["trip_id"] = df_new["trip_id"].astype(str)

# Filter invalid trip_ids
invalid_trip_ids = ["0", "Undetermined", "nan", "", "None"]
df_clean = df_new[~df_new["trip_id"].isin(invalid_trip_ids)].copy()


In [8]:
#✅ Step 3: Merge with trips.txt to Get route_id
df_with_trips = df_clean.merge(trips, on="trip_id", how="left")


In [9]:
#✅ Step 4: Merge with routes.txt to Get Descriptive Info
df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

df_full = df_with_trips.merge(routes, on="route_id", how="left")


In [10]:
#✅ Step 5: Create a data_quality Flag
def classify_row(row):
    if row["trip_id"] in invalid_trip_ids:
        return "invalid_trip_id"
    elif pd.isna(row["route_id"]):
        return "missing_route_id"
    elif pd.isna(row["route_long_name"]):
        return "missing_route_metadata"
    else:
        return "valid"

df_full["data_quality"] = df_full.apply(classify_row, axis=1)
print(df_full["data_quality"].value_counts())


data_quality
valid    7385
Name: count, dtype: int64


##✅ Validation Status: ✅ Complete
We’re now ready to proceed to the anomaly detection pipeline, which includes:

🔁 Processing Pipeline Overview
Here’s the sequence we’ll follow:

1. Set Detection Parameters
Distance threshold for "jump" (e.g., 250 meters)

Time threshold for "gap" (e.g., 90 seconds)

2. Prep and Sort Data
Keep only valid entries

Group by vehicle_id

Sort by timestamp

3. Calculate Jumps & Gaps
Use geodesic distance between consecutive points

Compute time delta between updates

4. Identify Anomalies
Mark as jump if distance > threshold

Mark as gap if time > threshold

Optionally flag sudden stop-to-start or heading reversals

5. Organize Output
Store anomalies with timestamps, vehicle IDs, severity

Merge context (route info, etc.)

6. Plot with Folium
Show map of vehicle trajectories

Color or mark anomalies (e.g., jumps as red lines, gaps as dashed)


##✅ Step 0: Define Parameters & Import Utilities
These values are based on:

Urban bus velocity norms (jump = teleport)

Expected GPS ping interval (<30s nominal)

Real-world practice in GTFS-RT anomaly detection

In [11]:
# Detection thresholds
DISTANCE_THRESHOLD_METERS = 250   # Vehicle "jump" threshold
TIME_THRESHOLD_SECONDS = 90       # Vehicle "gap" threshold


In [16]:
#Required Libraries
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.distance import geodesic

#from math import radians, cos, sin, asin, sqrt #Depricated; this was when using haversine


In [15]:
# #We’ll use a fast Haversine function for distance in meters:

# def haversine_meters(lat1, lon1, lat2, lon2):
#     # convert decimal degrees to radians
#     lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
#     # haversine formula
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
#     c = 2 * np.arcsin(np.sqrt(a))
#     return 6371000 * c  # Radius of Earth in meters


In [18]:
def parse_timestamp(ts):
    return pd.to_datetime(ts, utc=True)


###Optional geopy version

Reasoning: in areas with elevation shifts or nontrivial curvature (like Albuquerque’s foothills or valley transit zones), geopy.distance.geodesic provides more accurate spatial calculations than Haversine.

In [17]:
def geopy_distance(row1, row2):
    return geodesic(
        (row1["latitude"], row1["longitude"]),
        (row2["latitude"], row2["longitude"])
    ).meters


###Sample diagnostic
Try testing this with two consecutive points from one vehicle:

In [22]:
# Example: pick one vehicle's two points
vehicle_id = df_full[df_full["data_quality"] == "valid"]["vehicle_id"].iloc[0]
df_vehicle = df_full[df_full["vehicle_id"] == vehicle_id].sort_values("timestamp_collected")

# Check distance
geopy_distance(df_vehicle.iloc[0], df_vehicle.iloc[1])


348.71656092816204

##✅ Step 1: Prep and Sort the Valid Data
Let’s now build the full foundation for anomaly detection:

In [24]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 63


##✅ Step 2: Calculate Jumps and Gaps
We’ll now iterate through each vehicle's trajectory and compute:

Metric	How it's computed
- time_delta	Time difference between consecutive pings
- distance_m	Geodesic distance between GPS points
- is_jump	Distance > 250m
- is_gap	Time > 90s

In [25]:
# Store anomaly records
anomaly_records = []

for vehicle_id, group in vehicle_groups.items():
    group = group.sort_values("timestamp").reset_index(drop=True)

    for i in range(1, len(group)):
        row_prev = group.iloc[i - 1]
        row_curr = group.iloc[i]

        # Time delta
        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

        # Spatial distance (meters)
        dist = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters

        # Flag jump or gap
        is_jump = dist > DISTANCE_THRESHOLD_METERS
        is_gap = time_diff > TIME_THRESHOLD_SECONDS

        if is_jump or is_gap:
            anomaly_records.append({
                "vehicle_id": vehicle_id,
                "timestamp_prev": row_prev["timestamp"],
                "timestamp_curr": row_curr["timestamp"],
                "time_diff_sec": time_diff,
                "distance_m": dist,
                "is_jump": is_jump,
                "is_gap": is_gap,
                "lat_prev": row_prev["latitude"],
                "lon_prev": row_prev["longitude"],
                "lat_curr": row_curr["latitude"],
                "lon_curr": row_curr["longitude"],
                "route_short_name": row_curr.get("route_short_name", None),
                "route_long_name": row_curr.get("route_long_name", None),
            })


In [26]:
#✅ Convert Results to DataFrame
df_anomalies = pd.DataFrame(anomaly_records)
print("Total anomalies detected:", len(df_anomalies))
df_anomalies.head()


Total anomalies detected: 1880


,vehicle_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_jump,is_gap,lat_prev,lon_prev,lat_curr,lon_curr,route_short_name,route_long_name
0,354,2025-07-22 20:20:28.457332+00:00,2025-07-22 20:20:58.761413+00:00,30.304081,489.194027,True,False,35.080464,-106.622090,35.076054,-106.622117,None,Airport/Downtown
1,354,2025-07-22 20:22:29.430334+00:00,2025-07-22 20:22:59.635680+00:00,30.205346,400.874209,True,False,35.073222,-106.622092,35.069608,-106.622120,None,Airport/Downtown
2,354,2025-07-22 20:24:00.131520+00:00,2025-07-22 20:24:30.620138+00:00,30.488618,335.396112,True,False,35.067028,-106.622092,35.064006,-106.622027,None,Airport/Downtown
3,354,2025-07-22 20:25:00.837697+00:00,2025-07-22 20:25:31.095213+00:00,30.257516,375.283183,True,False,35.062173,-106.622060,35.058791,-106.622124,None,Airport/Downtown
4,354,2025-07-22 20:27:31.951960+00:00,2025-07-22 20:28:02.239893+00:00,30.287933,260.261488,True,False,35.054966,-106.622078,35.052620,-106.622047,None,Airport/Downtown


##Validation Summary:
| Metric                      | Value                                                        |
| --------------------------- | ------------------------------------------------------------ |
| **Total vehicles analyzed** | 63                                                           |
| **Total anomalies found**   | 1,880                                                        |
| **Sample anomalies**        | All from vehicle `354`, Airport/Downtown                     |
| **Common pattern**          | `True` jumps with `~30s` intervals and 260–490m displacement |

This suggests strong detection of micro-teleports — likely from missing intermediate GPS points or sudden location updates.

⚠️ Notable Data Quirk
"route_short_name": None
Yet "route_long_name" is present: "Airport/Downtown"

This likely means the route_short_name field wasn't carried through during one of your joins. If needed, we can retroactively fill it in using the routes.txt file.

###✅ Next Step: Filter and Organize Anomalies
Let’s prep for mapping by grouping and selecting:

In [27]:
#🔹 A. Filter to High-Severity Anomalies (Optional)
df_jumps = df_anomalies[df_anomalies["is_jump"]]
df_gaps = df_anomalies[df_anomalies["is_gap"]]

#You can also filter further by distance:
df_big_jumps = df_jumps[df_jumps["distance_m"] > 500]

#🔹 B. Group by Vehicle for Plotting
jump_groups = df_jumps.groupby("vehicle_id")


###Let's find some additional anomalies

🔹 3. Stationary-While-Moving (Stuck Vehicles)
Vehicle speed ≈ 0 for many updates in a row

Coordinates don’t change

Could indicate sensor freeze, or real-world delay

In [28]:
rolling_stationary = group["speed_mph"].rolling(window=4).mean() < 1.0


🔹 4. Impossible Speeds
Distance / time implies speed > 100 km/h

Implies faulty GPS update or sudden jump

In [29]:
df_anomalies["computed_speed_kph"] = (df_anomalies["distance_m"] / df_anomalies["time_diff_sec"]) * 3.6
df_anomalies[df_anomalies["computed_speed_kph"] > 120]


,vehicle_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_jump,is_gap,lat_prev,lon_prev,lat_curr,lon_curr,route_short_name,route_long_name,computed_speed_kph
518,455,2025-07-22 21:09:23.190301+00:00,2025-07-22 21:09:53.392474+00:00,30.202173,1113.136634,True,False,35.13711,-106.701018,35.144743,-106.69309,None,Coors,132.682237


🔹 5. Backtracking
Heading flips by ~180°

Or location regresses along route

Detection (heading-based):

In [ ]:
# delta_heading = abs(current_heading - previous_heading)
# flag if delta is near 180° or sudden direction reversal


🔹 6. Off-Route Movement
Vehicle deviates from known route shape / stop sequence

Requires shape.txt or external map of route paths

Detection:

Compare live lat/lon to shape or stop polyline

Buffer and check if point is "on route"

🔹 7. Repeated Points
Consecutive records have identical lat/lon and timestamps

Can indicate a stuck feed or backend error

In [30]:
(group["latitude"].diff().abs() < 0.00001) & (group["longitude"].diff().abs() < 0.00001)


,0
0,False
1,False
2,False
3,False
4,False
5,False
6,False
7,False
8,False
9,False


##Protyping additional anomalies

✅ Detection Function Set (for Each Vehicle Group)
Each function will accept a sorted DataFrame for a single vehicle (df_vehicle) and return either:

A list of anomaly rows

Or a new column appended to the same dataframe

🔹 1. Detect Jumps and Gaps

Already implemented, but here’s the modular form:

In [31]:
#1. Detect Jumps and Gaps
def detect_jumps_and_gaps(df, distance_threshold=250, time_threshold=90):
    anomalies = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters
        if time_diff > time_threshold or distance > distance_threshold:
            anomalies.append({
                "vehicle_id": row_curr["vehicle_id"],
                "timestamp_prev": row_prev["timestamp"],
                "timestamp_curr": row_curr["timestamp"],
                "time_diff_sec": time_diff,
                "distance_m": distance,
                "is_gap": time_diff > time_threshold,
                "is_jump": distance > distance_threshold,
                "anomaly_type": "jump_or_gap"
            })
    return anomalies


In [32]:
#🔹 2. Detect Stuck Vehicles
def detect_stuck_vehicle(df, speed_thresh=1.0, window=4):
    stuck_flags = (
        (df["speed_mph"].rolling(window).mean() < speed_thresh) &
        (df["latitude"].diff().abs().rolling(window).mean() < 0.0001) &
        (df["longitude"].diff().abs().rolling(window).mean() < 0.0001)
    )
    return df[stuck_flags.fillna(False)].assign(anomaly_type="stuck_vehicle")


In [33]:
#🔹 3. Detect Impossible Speeds
def detect_impossible_speeds(df, speed_limit_kph=120):
    records = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
        if time_diff == 0:
            continue
        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters
        speed_kph = (distance / time_diff) * 3.6
        if speed_kph > speed_limit_kph:
            records.append({
                "vehicle_id": row_curr["vehicle_id"],
                "timestamp_curr": row_curr["timestamp"],
                "computed_speed_kph": speed_kph,
                "distance_m": distance,
                "anomaly_type": "impossible_speed"
            })
    return pd.DataFrame(records)


In [34]:
#🔹 4. Detect Backtracking (Heading Reversal)
def detect_backtracking(df, reversal_thresh=160):
    backtrack_flags = df["heading"].diff().abs().between(reversal_thresh, 200)
    return df[backtrack_flags.fillna(False)].assign(anomaly_type="backtracking")


In [35]:
#🔹 5. Detect Repeated Points
def detect_repeated_points(df):
    repeated = (
        (df["latitude"].diff().abs() < 1e-5) &
        (df["longitude"].diff().abs() < 1e-5)
    )
    return df[repeated.fillna(False)].assign(anomaly_type="repeated_points")


In [36]:
#🔹 6. Detect Disappearance Without Return
def detect_disappeared(df, snapshot_end_time, min_gap_minutes=10):
    last_seen = df["timestamp"].max()
    if (snapshot_end_time - last_seen).total_seconds() > min_gap_minutes * 60:
        return pd.DataFrame([{
            "vehicle_id": df["vehicle_id"].iloc[0],
            "last_seen": last_seen,
            "anomaly_type": "disappearance"
        }])
    return pd.DataFrame()


In [37]:
#🔹 7. Detect Early Appearance
def detect_early_appearance(df, snapshot_start_time, margin_seconds=30):
    first_seen = df["timestamp"].min()
    if (first_seen - snapshot_start_time).total_seconds() < margin_seconds:
        return pd.DataFrame([{
            "vehicle_id": df["vehicle_id"].iloc[0],
            "first_seen": first_seen,
            "anomaly_type": "early_appearance"
        }])
    return pd.DataFrame()


###Detecting Off-Route Movement
Off-route movement is one of the most meaningful but also most complex GTFS-RT anomaly classes.

It occurs when:

A vehicle's reported location deviates significantly from its expected route shape as defined in shapes.txt.

Goal: Detect if a real-time GPS point is "off route" beyond a buffer (e.g. 50 meters) from its assigned shape_id.

####Requirements

To implement off-route detection, you'll need:

🔹 1. shapes.txt from static GTFS
This file defines the expected path of each shape_id as a polyline:

shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence

We use this to:

Reconstruct route geometry as a LineString (ordered path)

Assign each vehicle's trip to a shape

Measure distance from the live GPS point to that route line

🔹 2. Tools for Spatial Analysis
We recommend using:

pip install shapely geopandas

We'll use:

shapely.geometry.LineString – builds route paths

shapely.geometry.Point.distance – computes point-to-line distances

All in projected meters via GeoPandas if needed.

####Implementation Strategy

In [38]:
#1. Preprocess Route Shapes
import pandas as pd
from shapely.geometry import LineString

# Load and group shape points
shapes = pd.read_csv("/content/gtfs_static/shapes.txt")
shapes["shape_id"] = shapes["shape_id"].astype(str)

# Build LineStrings for each shape_id
shape_lines = {}
for shape_id, group in shapes.groupby("shape_id"):
    sorted_group = group.sort_values("shape_pt_sequence")
    coords = list(zip(sorted_group["shape_pt_lon"], sorted_group["shape_pt_lat"]))
    shape_lines[shape_id] = LineString(coords)


In [40]:
#2. Detect Off-Route for Each Vehicle Point
#We check if the GPS point is >50m from its assigned shape line.
from shapely.geometry import Point
from geopy.distance import geodesic

def detect_off_route(df_vehicle, shape_lines, buffer_m=50):
    records = []

    for _, row in df_vehicle.iterrows():
        shape_id = str(row.get("shape_id"))
        if shape_id not in shape_lines:
            continue  # Can't evaluate without shape geometry

        route_line = shape_lines[shape_id]
        vehicle_point = Point(row["longitude"], row["latitude"])
        projected_dist = route_line.distance(vehicle_point)

        # Convert degrees to meters (approximate)
        # Use geodesic for higher accuracy
        closest_point_on_route = route_line.interpolate(route_line.project(vehicle_point))
        route_point_coords = (closest_point_on_route.y, closest_point_on_route.x)
        vehicle_coords = (row["latitude"], row["longitude"])
        distance_m = geodesic(route_point_coords, vehicle_coords).meters

        if distance_m > buffer_m:
            records.append({
                "vehicle_id": row["vehicle_id"],
                "timestamp": row["timestamp"],
                "route_short_name": row.get("route_short_name"),
                "distance_from_route_m": distance_m,
                "anomaly_type": "off_route"
            })

    return pd.DataFrame(records)


✅ Step-by-Step Integration Plan

🧱 Pre-requisite: shapes.txt → shape_lines dictionary

You only need to run this once before processing vehicle groups:

In [41]:
import pandas as pd
from shapely.geometry import LineString

# Load shapes.txt
shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

# Build LineStrings for each shape_id
shape_lines = {}
for shape_id, group in shapes.groupby("shape_id"):
    sorted_group = group.sort_values("shape_pt_sequence")
    coords = list(zip(sorted_group["shape_pt_lon"], sorted_group["shape_pt_lat"]))
    shape_lines[shape_id] = LineString(coords)


In [47]:
#🔁 Function: Detect Off-Route Movement
#Add this detection function to your anomaly modules:
from shapely.geometry import Point
from geopy.distance import geodesic

def detect_off_route(df_vehicle, shape_lines, buffer_m=50):
    records = []
    for _, row in df_vehicle.iterrows():
        shape_id = str(row.get("shape_id"))
        if shape_id not in shape_lines:
            continue  # shape not known

        route_line = shape_lines[shape_id]
        vehicle_point = Point(row["longitude"], row["latitude"])

        # Find closest point on route and compute geodesic distance
        closest_point = route_line.interpolate(route_line.project(vehicle_point))
        dist_m = geodesic(
            (row["latitude"], row["longitude"]),
            (closest_point.y, closest_point.x)
        ).meters

        if dist_m > buffer_m:
            records.append({
              "vehicle_id": row["vehicle_id"],
              "timestamp": row["timestamp"],
              "route_short_name": row.get("route_short_name"),
              "distance_from_route_m": dist_m,
              "latitude": row["latitude"],
              "longitude": row["longitude"],
              "shape_id": shape_id,
              "anomaly_type": "off_route"
            })
    return pd.DataFrame(records)


#####🔄 Integration into Loop


In [48]:
#Assuming your vehicle groups are stored like:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#And you’re aggregating anomalies like:
all_anomalies = []

#Add off-route detection per vehicle:
for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

    # Call anomaly modules
    off_route_df = detect_off_route(df_vehicle, shape_lines)

    # Append results
    if not off_route_df.empty:
        all_anomalies.append(off_route_df)

# Final result
df_anomalies_offroute = pd.concat(all_anomalies, ignore_index=True)

#Optional Check:
print("Off-route anomalies detected:", len(df_anomalies_offroute))
df_anomalies_offroute.sort_values("distance_from_route_m", ascending=False).head()


Off-route anomalies detected: 225


,vehicle_id,timestamp,route_short_name,distance_from_route_m,latitude,longitude,shape_id,anomaly_type
153,628,2025-07-22 20:39:07.641420+00:00,None,1655.763689,35.123770,-106.62228,17192,off_route
154,628,2025-07-22 20:39:37.961321+00:00,None,1364.085044,35.123770,-106.61908,17192,off_route
152,628,2025-07-22 20:38:37.359784+00:00,None,1197.158035,35.128390,-106.61725,17192,off_route
151,628,2025-07-22 20:38:07.109451+00:00,None,1012.040833,35.129540,-106.61524,17192,off_route
116,467,2025-07-22 20:56:46.767304+00:00,None,945.883485,35.062921,-106.70293,17202,off_route


✅ Summary of Off-Route Detection

| Metric                        | Value                     |
| ----------------------------- | ------------------------- |
| **Total off-route anomalies** | 225                       |
| **Max distance from route**   | \~1.65 km (!!)            |
| **Common issue**              | `route_short_name = None` |


This confirms the module:

Correctly detects vehicles far outside their expected paths (e.g. 1+ km deviations)

Catches clear spatial mismatches (not false positives from small drifts)

Works even in topographically varied areas

###Additional steps

🔁 Strategy: shape_id → trip_id → route_id → route_short_name
You’ve already loaded:

trips.txt → contains trip_id, shape_id, and route_id

routes.txt → contains route_id, route_short_name

We’ll use these to build a mapping from shape_id → route_short_name.

In [49]:
#✅ Step 1: Build Mapping Table from Static GTFS
# Ensure consistent types
trips["shape_id"] = trips["shape_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

# Merge trips and routes to link shape_id to route_short_name
shape_route_map = (
    trips.merge(routes, on="route_id", how="left")
         .dropna(subset=["route_short_name"])
         .drop_duplicates(subset=["shape_id"])
         .set_index("shape_id")[["route_id", "route_short_name"]]
)


This gives a compact reference like:

| shape\_id | route\_id | route\_short\_name |
| --------- | --------- | ------------------ |
| 17192     | 54        | 8                  |


In [50]:
#✅ Step 2: Patch Missing route_short_name in Anomalies
# Ensure shape_id is string type
df_anomalies_offroute["shape_id"] = df_anomalies_offroute["shape_id"].astype(str)

# Join to enrich anomalies with route info
df_anomalies_offroute = (
    df_anomalies_offroute
    .merge(shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map"))
)

# If route_short_name was missing, replace it
df_anomalies_offroute["route_short_name"] = (
    df_anomalies_offroute["route_short_name"]
    .fillna(df_anomalies_offroute["route_short_name_from_map"])
)

# Drop helper column
df_anomalies_offroute = df_anomalies_offroute.drop(columns=["route_short_name_from_map"])


In [51]:
#Sanity Check
missing_routes = df_anomalies_offroute["route_short_name"].isnull().sum()
print(f"Remaining anomalies with missing route_short_name: {missing_routes}")
#If this prints 0, you’ve successfully patched all entries.

Remaining anomalies with missing route_short_name: 0


####✅ Step 1: Unify All Anomalies into df_anomalies_full
Assuming you’ve run detection modules and have these anomaly DataFrames:

df_anomalies_jumpgap – from detect_jumps_and_gaps(...)

df_anomalies_offroute – just finished

df_anomalies_stuck, df_anomalies_backtrack, df_anomalies_repeated, etc.

We’ll combine them:

In [53]:
# Run jump/gap detection and collect records
jumpgap_records = []

for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)
    anomalies = detect_jumps_and_gaps(df_vehicle)
    jumpgap_records.extend(anomalies)

# Convert to DataFrame
df_anomalies_jumpgap = pd.DataFrame(jumpgap_records)
print("Jump/Gap anomalies detected:", len(df_anomalies_jumpgap))


Jump/Gap anomalies detected: 1880


In [56]:
#Stuckvehicles
stuck_records = []
for vehicle_id, df_vehicle in vehicle_groups.items():
    df = detect_stuck_vehicle(df_vehicle)
    if not df.empty:
        stuck_records.append(df)

df_anomalies_stuck = pd.concat(stuck_records, ignore_index=True) if stuck_records else pd.DataFrame()


In [57]:
#Impossible Speeds
speed_records = []
for vehicle_id, df_vehicle in vehicle_groups.items():
    df = detect_impossible_speeds(df_vehicle)
    if not df.empty:
        speed_records.append(df)

df_anomalies_speed = pd.concat(speed_records, ignore_index=True) if speed_records else pd.DataFrame()


In [58]:
#Backtracking (heading reversals)
backtrack_records = []
for vehicle_id, df_vehicle in vehicle_groups.items():
    df = detect_backtracking(df_vehicle)
    if not df.empty:
        backtrack_records.append(df)

df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()


In [59]:
#Repeated Points
repeated_records = []
for vehicle_id, df_vehicle in vehicle_groups.items():
    df = detect_repeated_points(df_vehicle)
    if not df.empty:
        repeated_records.append(df)

df_anomalies_repeated = pd.concat(repeated_records, ignore_index=True) if repeated_records else pd.DataFrame()


In [60]:
#Disappearance Without Return
snapshot_end = df_full["timestamp"].max()

disappear_records = []
for vehicle_id, df_vehicle in vehicle_groups.items():
    df = detect_disappeared(df_vehicle, snapshot_end)
    if not df.empty:
        disappear_records.append(df)

df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()


KeyError: 'timestamp'

In [54]:
# List of all collected anomaly frames
anomaly_frames = [
    df_anomalies_jumpgap,     # Already structured with 'anomaly_type'
    df_anomalies_offroute,
    df_anomalies_stuck,
    df_anomalies_backtrack,
    df_anomalies_repeated,
    df_anomalies_disappear,
    df_anomalies_early,
    df_anomalies_speed
]

# Filter out any that don't exist or are empty
anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]

# Combine
df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True)
print("Total anomalies:", len(df_anomalies_full))


NameError: name 'df_anomalies_stuck' is not defined

In [62]:
from datetime import datetime

# Step 1: Normalize timestamp and rebuild vehicle groups
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

# Step 2: Detect each anomaly type

jumpgap_records = []
stuck_records = []
speed_records = []
backtrack_records = []
repeated_records = []
disappear_records = []
early_records = []
offroute_records = []

snapshot_start = df_valid["timestamp"].min()
snapshot_end = df_valid["timestamp"].max()

for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

    # 1. Jumps and Gaps
    jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle))

    # 2. Stuck Vehicles
    stuck = detect_stuck_vehicle(df_vehicle)
    if not stuck.empty:
        stuck_records.append(stuck)

    # 3. Impossible Speeds
    speed = detect_impossible_speeds(df_vehicle)
    if not speed.empty:
        speed_records.append(speed)

    # 4. Backtracking
    backtrack = detect_backtracking(df_vehicle)
    if not backtrack.empty:
        backtrack_records.append(backtrack)

    # 5. Repeated Points
    repeat = detect_repeated_points(df_vehicle)
    if not repeat.empty:
        repeated_records.append(repeat)

    # 6. Disappearance
    disappear = detect_disappeared(df_vehicle, snapshot_end)
    if not disappear.empty:
        disappear_records.append(disappear)

    # 7. Early Appearance
    early = detect_early_appearance(df_vehicle, snapshot_start)
    if not early.empty:
        early_records.append(early)

    # 8. Off-Route
    offroute = detect_off_route(df_vehicle, shape_lines)
    if not offroute.empty:
        offroute_records.append(offroute)

# Step 3: Combine to DataFrames
df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True) if stuck_records else pd.DataFrame()
df_anomalies_speed     = pd.concat(speed_records, ignore_index=True) if speed_records else pd.DataFrame()
df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True) if repeated_records else pd.DataFrame()
df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
df_anomalies_early     = pd.concat(early_records, ignore_index=True) if early_records else pd.DataFrame()
df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True) if offroute_records else pd.DataFrame()

# Step 4: Combine all anomalies into a single DataFrame
anomaly_frames = [
    df_anomalies_jumpgap,
    df_anomalies_stuck,
    df_anomalies_speed,
    df_anomalies_backtrack,
    df_anomalies_repeated,
    df_anomalies_disappear,
    df_anomalies_early,
    df_anomalies_offroute
]

anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]
df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True)
print("Unified anomaly count:", len(df_anomalies_full))
df_anomalies_full["anomaly_type"].value_counts()


Unified anomaly count: 5428


,count
anomaly_type,
repeated_points,1930
jump_or_gap,1880
stuck_vehicle,1286
off_route,225
early_appearance,62
backtracking,44
impossible_speed,1
